In [1]:
from rdflib_r2r import R2RStore, R2RMapping
from sqlalchemy import create_engine
from pathlib import Path
import rdflib
from rdflib.compare import to_isomorphic, graph_diff

echo = False

def create_database(db, path, *sql_fnames):
    for sql_fname in sql_fnames:
        sql_script = (path / sql_fname).open().read()
        conn = db.raw_connection()
        try:
            if hasattr(conn.connection, "c"):
                cursor = conn.connection.c
            else:
                cursor = conn.cursor()
            cursor.execute(sql_script, (), None)
        except Exception as e:
            try:
                cursor = conn.cursor()
                cursor.executescript(sql_script)
            except Exception as e:
                raise Exception(f"Problem with {path}: {e}")
            finally:
                conn.close()
        finally:
            conn.close()

# db = create_engine(
#     "duckdb:///:memory:",
#     echo=echo,
#     # future=True,
#     connect_args={"read_only": False},
# )
# db.dialect.server_version_info = (0,)

import sqlite3
db = create_engine(
    "sqlite:///:memory:",
    echo=echo,
    future=True, 
    connect_args={
        "detect_types": sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES,
        'timeout': 30 * 1000
    },
    native_datetime=False, # breaks rdb2rdf test D016
    # native_datetime=True, # breaks BSBM test 3
)

# path = Path('tests/rdb2rdf-ts/D014-3tables1primarykey1foreignkey') # 14b
# path = Path('tests/rdb2rdf-ts/D000-1table1column0rows')
path = Path('tests/rdb2rdf-ts/D001-1table1column1row')
sql_fname = 'create.sql'

sql_script = (path / sql_fname).open().read()
create_database(db, path, sql_fname)

# Loads R2RML mapping
# mapfile = path / 'r2rmlb.ttl'
# fmt = rdflib.util.guess_format(str(mapfile))
# mapping = R2RMapping(rdflib.Graph().parse(str(mapfile), format=fmt))

# Creates Direct Mapping from DB schema
mapping = R2RMapping.from_db(db)

g_made = rdflib.Graph(R2RStore(db=db, mapping=mapping))
print(to_isomorphic(g_made).serialize())

#print(g_made.store.getSQL("SELECT * WHERE {?s ?p ?o}"))
query = g_made.store.getSQL("SELECT * WHERE {?s ?p ?o}")
db.raw_connection().cursor().execute(query).fetchall()

/var/folders/jd/y6jxdm1s23l3qtl8w9js19880000gn/T/ipykernel_4907/3444967154.py:14: SADeprecationWarning: The _ConnectionFairy.connection attribute is deprecated; please use 'driver_connection' (deprecated since: 2.0)
  if hasattr(conn.connection, "c"):
@base <http://example.com/base/> .
@prefix ns1: <http://www.w3.org/ns/r2rml#> .

[] a ns1:TriplesMap ;
    ns1:logicalTable [ ns1:tableName "\"Student\"" ] ;
    ns1:predicateObjectMap [ ns1:objectMap [ ns1:column "\"Name\"" ] ;
            ns1:predicate <Student#Name> ] ;
    ns1:subjectMap [ ns1:class <Student> ;
            ns1:termType ns1:BlankNode ] .




@prefix ns1: <http://example.com/base/Student#> .

[] a <http://example.com/base/Student> ;
    ns1:Name "Venus" .




[('_:Student#1',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://example.com/base/Student>'),
 ('_:Student#1', '<http://example.com/base/Student#Name>', 'Venus')]

In [3]:
# 模版
from sqlalchemy import text

with db.connect() as conn:
    for r in conn.execute(text(g_made.store.getSQL("SELECT * WHERE {?s ?p ?o}"))):
        print(r)

('_:Student#1', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://example.com/base/Student>')
('_:Student#1', '<http://example.com/base/Student#Name>', 'Venus')


In [36]:
# D001
# with db.connect() as conn:
# #     result = conn.execute(text("""SELECT rowid, Name FROM Student"""))
#     result = conn.execute(text("""
#     SELECT CAST(:param_1 AS VARCHAR) || CAST(CAST("Student".rowid AS VARCHAR) AS VARCHAR) AS s, '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' AS p, '<http://example.com/base/Student>' AS o, NULL AS g
#     FROM "Student" UNION ALL SELECT CAST(:param_2 AS VARCHAR) || CAST(CAST("Student".rowid AS VARCHAR) AS VARCHAR) AS s, '<http://example.com/base/Student#Name>' AS p, "Student"."Name" AS o, NULL AS g
#     FROM "Student"
#     """), {"param_1":"_:Student#", "param_2":"_:Student#"})
#     display(result.all())

In [5]:
with db.connect() as conn:
    result = conn.execute(text("""
    SELECT '_:Student#' || CAST(CAST("Student".rowid AS VARCHAR) AS VARCHAR) AS s,
                    '<http://example.com/base/Student#Name>' AS p,
                    "Student"."Name" AS o,
                    NULL AS g
   FROM "Student"
    """),{})
    display(result.all())

[('_:Student#1', '<http://example.com/base/Student#Name>', 'Venus', None)]

In [3]:
for t in g_made:
    print(t)

/Users/vickyli117/Desktop/rdflib-r2r/rdflib_r2r/r2r_store.py:652: SAWarning: SELECT statement has a cartesian product between FROM element(s) "DEPT_ref" and FROM element "EMP".  Apply join condition(s) between each element to resolve.
  rows = list(conn.execute(query))


(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/base/DEPT'))
(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://example.com/base/DEPT#dname'), rdflib.term.Literal('APPSERVER'))
(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://example.com/base/DEPT#loc'), rdflib.term.Literal('NEW YORK'))
(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://example.com/base/DEPT#deptno'), rdflib.term.Literal('10', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/base/EMP'))
(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URIRef('http://example.com/base/EMP#job'), rdflib.term.Literal('CLERK'))
(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URIRef('h

In [4]:
print(to_isomorphic(g_made).serialize())

/Users/vickyli117/Desktop/rdflib-r2r/rdflib_r2r/r2r_store.py:652: SAWarning: SELECT statement has a cartesian product between FROM element(s) "EMP" and FROM element "DEPT_ref".  Apply join condition(s) between each element to resolve.
  rows = list(conn.execute(query))


@prefix ns1: <http://example.com/base/EMP#> .
@prefix ns2: <http://example.com/base/LIKES#> .
@prefix ns3: <http://example.com/base/DEPT#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/base/EMP/empno=7369> a <http://example.com/base/EMP> ;
    ns1:deptno 10 ;
    ns1:empno 7369 ;
    ns1:ename "SMITH" ;
    ns1:etype "PART_TIME" ;
    ns1:job "CLERK" ;
    ns1:ref-deptno [ a <http://example.com/base/DEPT> ;
            ns3:deptno 10 ;
            ns3:dname "APPSERVER" ;
            ns3:loc "NEW YORK" ] .

[] a <http://example.com/base/LIKES> ;
    ns2:id 7369 ;
    ns2:likeType "Playing" ;
    ns2:likedObj "Soccer" .

[] a <http://example.com/base/LIKES> ;
    ns2:id 7369 ;
    ns2:likeType "Watching" ;
    ns2:likedObj "Basketball" .




In [6]:
outfile = path / 'directGraph.ttl'
# outfile = path / 'mappeda.nq'

fmt = rdflib.util.guess_format(str(outfile)) 
print(rdflib.Graph().parse(str(outfile), format=fmt).serialize())

for t in rdflib.Graph().parse(str(outfile), format=fmt):
    print(t)

@prefix ns1: <http://example.com/base/EMP#> .
@prefix ns2: <http://example.com/base/LIKES#> .
@prefix ns3: <http://example.com/base/DEPT#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/base/EMP/empno=7369> a <http://example.com/base/EMP> ;
    ns1:deptno 10 ;
    ns1:empno 7369 ;
    ns1:ename "SMITH" ;
    ns1:etype "PART_TIME" ;
    ns1:job "CLERK" ;
    ns1:ref-deptno [ a <http://example.com/base/DEPT> ;
            ns3:deptno 10 ;
            ns3:dname "APPSERVER" ;
            ns3:loc "NEW YORK" ] .

[] a <http://example.com/base/LIKES> ;
    ns2:id 7369 ;
    ns2:likeType "Playing" ;
    ns2:likedObj "Soccer" .

[] a <http://example.com/base/LIKES> ;
    ns2:id 7369 ;
    ns2:likeType "Watching" ;
    ns2:likedObj "Basketball" .


(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URIRef('http://example.com/base/EMP#empno'), rdflib.term.Literal('7369', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rd